# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [15]:
%matplotlib inline
import pandas as pd
import numpy as np
import math as math
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [16]:
dados= pd.read_excel('tweets_samsung_classificados.xlsx')
dados.head(4)

,Treinamento,relevancia,positividade
0,@samsungbrasil j5 prime samsung. meu cartão ai...,0,1
1,minha tia falando que não troca samsung por ap...,1,0
2,rt @beatriz5correia: pls rt para encontrar o m...,0,0
3,@michaelserra aqui só se for com rom customiza...,1,0


In [17]:
index=0
for frase in dados["Treinamento"]:
    if index<=299:
        frase_new=re.split(" ", frase)
        dados["Treinamento"][index]=frase_new
        index+=1

C:\Users\gi_gi\Anaconda3\ciência dos dados\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [21]:
import nltk

print(string.punctuation)
#remove ponctuation from each word 

import string 
table = str.maketrans('', '', string.punctuation)
sem_pontuacao=[w.translate(table) for w in dados["Treinamento"][index]]
print(sem_pontuacao[:100])


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


KeyError: 300

In [28]:
index=0
palavras_neg=[]
palavras_pos=[]
palavras_irrel=[]
palavras_rel=[]
for e in dados["relevancia"]:
    if e==1: 
        for i in dados["Treinamento"][index]:
            palavras_rel.append(i)
        if dados["positividade"][index]==0:
             for i in dados["Treinamento"][index]:
                palavras_neg.append(i)
        else:
             for i in dados["Treinamento"][index]:
                palavras_pos.append(i)
    else:
        for i in dados["Treinamento"][index]:
            palavras_irrel.append(i)
    index+=1

soma=0
for e in palavras_neg:
    if "trava" in e:
        soma+=1
soma



23

In [29]:
#for e in dados["Treinamento"]:
    #prob_rel=1
    #for palavras in e:
     #   soma=0
      #  for item in palavras_rel:
       #     if palavra in item:
        #        soma+=1
        #prob_rel*=(soma/len(palavras_rel))
    #prob_rel*##numero da probabilidadae de ser relevante 

In [30]:
# probabilidade da palavra ser positiva dentre as palavras relevantes
p_pos=len(palavras_pos)/len(palavras_rel)

# probabilidade da palavra ser negativa dentre as palavras relevantes
p_neg=len(palavras_neg)/len(palavras_rel)

# probabilidade da palavra ser relevante dentre todas as palavras
p_rel=len(palavras_rel)/(len(palavras_rel)+len(palavras_irrel))

# probabilidade da palavra ser irrelevante dentre todas as palavras
p_irrel=len(palavras_irrel)/(len(palavras_rel)+len(palavras_irrel))

print("Probabilidade de ser positiva: {:.2f}%".format(p_pos*100))
print("Probabilidade de ser negativa: {:.2f}%".format(p_neg*100))
print("Probabilidade de ser relevante: {:.2f}%".format(p_rel*100))
print("Probabilidade de ser irrelevante: {:.2f}%".format(p_irrel*100))

Probabilidade de ser positiva: 37.87%
Probabilidade de ser negativa: 62.13%
Probabilidade de ser relevante: 52.75%
Probabilidade de ser irrelevante: 47.25%


In [31]:
def probabilidade(frase, palavras, num_palavras, prob, l, alpha=1): #alpha é o fator do laplace smoothing
    probabilidade2 = log(prob)
    

    total = num_palavras + alpha*l
    for p in dados["Treinamento"][index]:
        vezes = alpha
        if p in palavras:
            vezes += palavras[p]
        probabilidade2 += log(vezes/total)
            
    return probabilidade2

In [32]:
def comaparando(prob_0, prob_1):

    if prob_0 > prob_1:
        return 0
    elif prob_1 > prob_0:
        return 1

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
